<a target="_blank" href="https://colab.research.google.com/github/TransformerLensOrg/TransformerLens/blob/main/demos/Attribution_Patching_Demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

 # Attribution Patching Demo
 **Read [the accompanying blog post here](https://neelnanda.io/attribution-patching) for more context**
 This is an interim research report, giving a whirlwind tour of some unpublished work I did at Anthropic (credit to the then team - Chris Olah, Catherine Olsson, Nelson Elhage and Tristan Hume for help, support, and mentorship!)

 The goal of this work is run activation patching at an industrial scale, by using gradient based attribution to approximate the technique - allow an arbitrary number of patches to be made on two forwards and a single backward pass

 I have had less time than hoped to flesh out this investigation, but am writing up a rough investigation and comparison to standard activation patching on a few tasks to give a sense of the potential of this approach, and where it works vs falls down.

 <b style="color: red">To use this notebook, go to Runtime > Change Runtime Type and select GPU as the hardware accelerator.</b>

 **Tips for reading this Colab:**
 * You can run all this code for yourself!
 * The graphs are interactive!
 * Use the table of contents pane in the sidebar to navigate
 * Collapse irrelevant sections with the dropdown arrows
 * Search the page using the search in the sidebar, not CTRL+F

 ## Setup (Ignore)

In [ ]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
import os

DEBUG_MODE = False
IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"
try:
  import google.colab

  IN_COLAB = True
  print("Running as a Colab notebook")
except:
  IN_COLAB = False
  print("Running as a Jupyter notebook - intended for development only!")
  from IPython import get_ipython

  ipython = get_ipython()
  # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
  ipython.magic("load_ext autoreload")
  ipython.magic("autoreload 2")

if IN_COLAB or IN_GITHUB:
  %pip install transformer_lens
  %pip install torchtyping
  # Install my janky personal plotting utils
  %pip install git+https://github.com/neelnanda-io/neel-plotly.git
  # Install another version of node that makes PySvelte work way faster
  %pip install circuitsvis
  # Needed for PySvelte to work, v3 came out and broke things...
  %pip install typeguard==2.13.3

Running as a Colab notebook
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.13.3
    Uninstalling typeguard-2.13.3:
      Successfully uninstalled typeguard-2.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtyping 0.1.5 requires typeguard<3,>=2.11.1, but you have typeguard 4.4.4 which is incompatible.
  Using cached typeguard-2.13.3-py3-none-any.whl.metadata (3.6 kB)
Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.4
    Uninstalling typeguard-4.4.4:
      Successfully uninstalled typeguard-4.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformer-lens 2.16.1 requires typeguard<5.0,>=4.2, but you have t

In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
  # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
  pio.renderers.default = "colab"
else:
  pio.renderers.default = "notebook_connected"

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from torchtyping import TensorType as TT
from typing import List, Union, Optional, Callable
from functools import partial
import copy
import itertools
import json

from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML, Markdown

In [ ]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import (
    HookedTransformer,
    HookedTransformerConfig,
    FactoredMatrix,
    ActivationCache,
)

 Plotting helper functions from a janky personal library of plotting utils. The library is not documented and I recommend against trying to read it, just use your preferred plotting library if you want to do anything non-obvious:

In [ ]:
from neel_plotly import line, imshow, scatter

In [ ]:
import transformer_lens.patching as patching

 ## IOI Patching Setup
 This just copies the relevant set up from Exploratory Analysis Demo, and isn't very important.

In [ ]:
model = HookedTransformer.from_pretrained("gpt2-small")
model.set_use_attn_result(True)
model.cfg.use_attn_in = True
model.cfg.use_hook_mlp_in = True

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


In [ ]:
prompts = [
    "When John and Mary went to the shops, John gave the bag to",
    "When John and Mary went to the shops, Mary gave the bag to",
    "When Tom and James went to the park, James gave the ball to",
    "When Tom and James went to the park, Tom gave the ball to",
    "When Dan and Sid went to the shops, Sid gave an apple to",
    "When Dan and Sid went to the shops, Dan gave an apple to",
    "After Martin and Amy went to the park, Amy gave a drink to",
    "After Martin and Amy went to the park, Martin gave a drink to",
]
answers = [
    (" Mary", " John"),
    (" John", " Mary"),
    (" Tom", " James"),
    (" James", " Tom"),
    (" Dan", " Sid"),
    (" Sid", " Dan"),
    (" Martin", " Amy"),
    (" Amy", " Martin"),
]

In [ ]:
# Tokenize the clean prompts
clean_tokens = model.to_tokens(prompts)

# Swap each adjacent pair, with a hacky list comprehension

# Given prompts A, B, C, D, the corrupted tokens will be B, A, D, C
# Creates a dataset where the indirect object is not the expected answer based on the origina prompt
corrupted_tokens = clean_tokens[
    [(i + 1 if i % 2 == 0 else i - 1) for i in range(len(clean_tokens))]
]

# Prints almost the same sequence but the subject is swapped
print("Clean string 0", model.to_string(clean_tokens[0]))
print("Corrupted string 0", model.to_string(corrupted_tokens[0]))

# These are the 2 answer choices for each prompt as tokens
answer_token_indices = torch.tensor(
    [
        [model.to_single_token(answers[i][j]) for j in range(2)]
        for i in range(len(answers))
    ],
    device=model.cfg.device,
)
print("Answer token indices: \n", answer_token_indices)

Clean string 0 <|endoftext|>When John and Mary went to the shops, John gave the bag to
Corrupted string 0 <|endoftext|>When John and Mary went to the shops, Mary gave the bag to
Answer token indices: 
 tensor([[ 5335,  1757],
        [ 1757,  5335],
        [ 4186,  3700],
        [ 3700,  4186],
        [ 6035, 15686],
        [15686,  6035],
        [ 5780, 14235],
        [14235,  5780]], device='cuda:0')


In [ ]:
def get_logit_diff(logits, answer_token_indices=answer_token_indices):
  # get_logit_diff quantifies the amount the model prefers the correct answer token over the incorrect one

  # If the full sequence of logits are passed in, only use the final logits for the prediction of the next token
  if len(logits.shape) == 3:
    # Get final logits only
    logits = logits[:, -1, :]

  # Extracts the logits corresponding to the correct answer token for each prompts
  correct_logits = logits.gather(1, answer_token_indices[:, 0].unsqueeze(1))

  # Extracts the logits corresponding to the incorrect answer token for each prompts
  incorrect_logits = logits.gather(1, answer_token_indices[:, 1].unsqueeze(1))

  # Averages the difference between the correct and incorrect logits across the entire batch
  return (correct_logits - incorrect_logits).mean()

  # A higher difference value means that the model is more confident in the correct answer

# Get the clean output logits and the intermediate activations
clean_logits, clean_cache = model.run_with_cache(clean_tokens)

# Get the corrupted output logits and the intermediate activations
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)

# The model is more confident in the clean answer
clean_logit_diff = get_logit_diff(clean_logits, answer_token_indices).item()
print(f"Clean logit diff: {clean_logit_diff:.4f}")

# The model is less confident in the original clean answer
corrupted_logit_diff = get_logit_diff(corrupted_logits, answer_token_indices).item()
print(f"Corrupted logit diff: {corrupted_logit_diff:.4f}")

Clean logit diff: 3.5519
Corrupted logit diff: -3.5519


In [ ]:
CLEAN_BASELINE = clean_logit_diff
CORRUPTED_BASELINE = corrupted_logit_diff

# Normalizes the logit difference
def ioi_metric(logits, answer_token_indices=answer_token_indices):
  return (get_logit_diff(logits, answer_token_indices) - CORRUPTED_BASELINE) / (
      CLEAN_BASELINE - CORRUPTED_BASELINE
  )

# A value of 1 means that the output is the same as the clean baseline
print(f"Clean Baseline is 1: {ioi_metric(clean_logits).item():.4f}")

# A value of 0 means that the output is the same as the corrupted baseline
print(f"Corrupted Baseline is 0: {ioi_metric(corrupted_logits).item():.4f}")

Clean Baseline is 1: 1.0000
Corrupted Baseline is 0: 0.0000


## Patching
 In the following cells, we define attribution patching and use it in various ways on the model.

In [ ]:
# Defines a type hint that takes a tensor of shape (batch_and_pos_dims, d_model) which returns a single float
Metric = Callable[[TT["batch_and_pos_dims", "d_model"]], float]

In [ ]:
# Lambda function that checks if _input is not in the name
filter_not_qkv_input = lambda name: "_input" not in name
# Used later to decide which activations to cache

def get_cache_fwd_and_bwd(model, tokens, metric):
  # Removes hooks from previous runs
  model.reset_hooks()

  # cache stores the forward pass activations
  cache = {}

  # When this hook function is triggered during the forward pass, it gets the activation and stores a detached copy in the cache
  def forward_cache_hook(act, hook):
    cache[hook.name] = act.detach()

  # Adds the hook to the model during the forward pass for non inputs
  model.add_hook(filter_not_qkv_input, forward_cache_hook, "fwd")

  # grad_cache stores the backward pass gradients
  grad_cache = {}

  # When this hook function is triggered during the backward pass, it gets the gradients and stores a detached copy in the cache
  def backward_cache_hook(act, hook):
    grad_cache[hook.name] = act.detach()

  # Adds the hook to the model during the backward pass for non inputs
  model.add_hook(filter_not_qkv_input, backward_cache_hook, "bwd")

  # Uses the metric function (IOI metric) and backpropagates

  # Forward pass that caches the model's activations
  value = metric(model(tokens))

  # Backward pass that caches the model's gradients that are backpropagated from the metric function
  value.backward()
  model.reset_hooks()

  return (
      value.item(), # Numerical value of the calculated metric
      ActivationCache(cache, model), # Cached forward pass activations
      ActivationCache(grad_cache, model), # Cached backward pass activations
  )

# Get the metric and caches for the clean prompts
clean_value, clean_cache, clean_grad_cache = get_cache_fwd_and_bwd(
    model, clean_tokens, ioi_metric
)

print("Clean Value:", clean_value)
print("Clean Activations Cached (Fwd Pass):", len(clean_cache))
print("Clean Gradients Cached (Bwd Pass):", len(clean_grad_cache))

# Get the metric and caches for the corrupted prompts
corrupted_value, corrupted_cache, corrupted_grad_cache = get_cache_fwd_and_bwd(
    model, corrupted_tokens, ioi_metric
)

print("Corrupted Value:", corrupted_value)
print("Corrupted Activations Cached (Fwd Pass):", len(corrupted_cache))
print("Corrupted Gradients Cached (Bwd Pass):", len(corrupted_grad_cache))

Clean Value: 1.0
Clean Activations Cached (Fwd Pass): 244
Clean Gradients Cached (Bwd Pass): 244
Corrupted Value: 0.0
Corrupted Activations Cached (Fwd Pass): 244
Corrupted Gradients Cached (Bwd Pass): 244


 ### Attention Attribution
 The easiest thing to start with is to not even engage with the corrupted tokens/patching, but to look at the attribution of the attention patterns - that is, the linear approximation to what happens if you set each element of the attention pattern to zero. This, as it turns out, is a good proxy to what is going on with each head!
 Note that this is *not* the same as what we will later do with patching. In particular, this does not set up a careful counterfactual! It's a good tool for what's generally going on in this problem, but does not control for eg stuff that systematically boosts John > Mary in general, stuff that says "I should activate the IOI circuit", etc. Though using logit diff as our metric *does*
 Each element of the batch is independent and the metric is an average logit diff, so we can analyse each batch element independently here. We'll look at the first one, and then at the average across the whole batch (note - 4 prompts have indirect object before subject, 4 prompts have it the other way round, making the average pattern harder to interpret - I plot it over the first sequence of tokens as a mildly misleading reference).
 We can compare it to the interpretability in the wild diagram, and basically instantly recover most of the circuit!

In [ ]:
def create_attention_attr(
    clean_cache, clean_grad_cache
) -> TT["batch", "layer", "head_index", "dest", "src"]:
  # Evaluates the most important parts of the attention patterns to the IOI metric based on a linear approximation

  # Extracts the attention patterns from the clean activation cache for each layer
  # Represents the weights given to each source token
  attention_stack = torch.stack(
      [clean_cache["pattern", l] for l in range(model.cfg.n_layers)], dim=0
  )

  # Extracts the attention patterns from the clean gradient cache for each layer
  # Represents the amount the final metric would change given a small change to each element of the attention pattern
  attention_grad_stack = torch.stack(
      [clean_grad_cache["pattern", l] for l in range(model.cfg.n_layers)], dim=0
  )

  # Linear approximatino of the contribution of each element in the attention pattern to the final metric
  attention_attr = attention_grad_stack * attention_stack
  attention_attr = einops.rearrange(
      attention_attr,
      "layer batch head_index dest src -> batch layer head_index dest src",
  )

  return attention_attr

# Calculates a value for each element in the attention patterns of the clean run that approximates how much the specific attention weight contributes to the final IOI metric

# This linear approximation doesn't involve counterfactuals like patching, but it serves as a quick way to identify potentially important attention patterns and heads
attention_attr = create_attention_attr(clean_cache, clean_grad_cache)

In [ ]:
attention_attr.shape # shape: (batch_size=8, layers=12, heads=12, dest_tokens=15, src_tokens=15)

torch.Size([8, 12, 12, 15, 15])

In [ ]:
HEAD_NAMES = [
    f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)
]
HEAD_NAMES_SIGNED = [f"{name}{sign}" for name in HEAD_NAMES for sign in ["+", "-"]]
HEAD_NAMES_QKV = [
    f"{name}{act_name}" for name in HEAD_NAMES for act_name in ["Q", "K", "V"]
]

print(HEAD_NAMES[:5])
print(HEAD_NAMES_SIGNED[:5])
print(HEAD_NAMES_QKV[:5])

['L0H0', 'L0H1', 'L0H2', 'L0H3', 'L0H4']
['L0H0+', 'L0H0-', 'L0H1+', 'L0H1-', 'L0H2+']
['L0H0Q', 'L0H0K', 'L0H0V', 'L0H1Q', 'L0H1K']


 An extremely janky way to plot the attention attribution patterns. We scale them to be in [-1, 1], split each head into a positive and negative part (so all of it is in [0, 1]), and then plot the top 20 head-halves (a head can appear twice!) by the max value of the attribution pattern.

In [ ]:
from circuitsvis.attention import attention_heads

Overall, the propose of this visualization is to get a quick, linear approximation of which parts of the model's attention mechanism are most responsible for the difference between the clean and corrupted outputs on the IOI task. Specifically, it shows how much each element in the attention pattern contributes to the final IOI metric.

**Results:** The viusalized heatmap shows red for positive and blue for negative to indicate the magnitude of the attribution.

A positive attribution value (red) means that if that specific attention weight were increased, it would incresae the IOI metric (making the model more likely to predict the correct name).

A negative attribution value (blue) means that if that specific attention weight were increased, it would decrease the IOI metric (making the model less likely to predict the correct name).

The **destination token** corresponds to the **Query (Q) vector** in Attention. This represents the token that is being attended to. It is on the vertical axis.

The **source token** corresponds to the **Key (K) vector** in Attention. This represents the token that is being attended from. It is on the horizontal axis.

In the results, we se that at head 14, there are many blue squares. For example, a blue square at (destination position "John", and source position "When") means that this head attending from "When" to "John" has a negative impact on the IOI metric.

In [ ]:
def plot_attention_attr(attention_attr, tokens, top_k=20, index=0, title=""):
    if len(tokens.shape) == 2:
      tokens = tokens[index]

    if len(attention_attr.shape) == 5:
      attention_attr = attention_attr[index]

    # Clamp positive and negative attribution values
    # Allows visualization of the positive and negative contributions of each attention weight separately
    attention_attr_pos = attention_attr.clamp(min=-1e-5)
    attention_attr_neg = -attention_attr.clamp(max=1e-5)

    # Stack the positive and negative attribution components
    # The new dimension separates them
    attention_attr_signed = torch.stack([attention_attr_pos, attention_attr_neg], dim=0)
    attention_attr_signed = einops.rearrange(
        attention_attr_signed,
        "sign layer head_index dest src -> (layer head_index sign) dest src",
    )

    # Normalizes the values to be between [0, 1] for visualization
    attention_attr_signed = attention_attr_signed / attention_attr_signed.max()

    # Finds the indices of the heads with the largest attribution values

    # Finds the maximum value across the src and dest dims for each signed head
    # Sorts the maximum values in descending order to get the indices of the top contributing signed heads
    attention_attr_indices = (
        attention_attr_signed.max(-1).values.max(-1).values.argsort(descending=True)
    )

    # print(attention_attr_indices.shape)
    # print(attention_attr_indices)

    attention_attr_signed = attention_attr_signed[attention_attr_indices, :, :]
    head_labels = [HEAD_NAMES_SIGNED[i.item()] for i in attention_attr_indices]

    if title:
      display(Markdown("### " + title))

    # Visualize the attention attribution data
    rendered = attention_heads(
      attention=attention_attr_signed.permute(1, 2, 0).detach().cpu().numpy(),
      tokens=model.to_str_tokens(tokens),
      attention_head_names=[f"Head {i}" for i in range(attention_attr_signed.shape[0])]
    )

    display(rendered)

# plot the attention attribution for the first sequence in the batch
plot_attention_attr(
    attention_attr,
    clean_tokens,
    index=0,
    title="Attention Attribution for first sequence",
)

# Plot the attention attribution summed across all sequences in the batch
# Gives an overall sense of which attention patterns are most important on average
plot_attention_attr(
    attention_attr.sum(0),
    clean_tokens[0],
    title="Summed Attention Attribution for all sequences",
)
print(
    "Note: Plotted over first sequence for reference, but pairs have IO and S1 in different positions."
)

### Attention Attribution for first sequence

### Summed Attention Attribution for all sequences

Note: Plotted over first sequence for reference, but pairs have IO and S1 in different positions.


 ## Attribution Patching
 In the following sections, I will implement various kinds of attribution patching, and then compare them to the activation patching patterns (activation patching code copied from [Exploratory Analysis Demo](https://neelnanda.io/exploratory-analysis-demo))
 ### Residual Stream Patching
 <details><summary>Note: We add up across both d_model and batch (Explanation).</summary>
 We add up along d_model because we're taking the dot product - the derivative *is* the linear map that locally linearly approximates the metric, and so we take the dot product of our change vector with the derivative vector. Equivalent, we look at the effect of changing each coordinate independently, and then combine them by adding it up - it's linear, so this totally works.
 We add up across batch because we're taking the average of the metric, so each individual batch element provides `1/batch_size` of the overall effect. Because each batch element is independent of the others and no information moves between activations for different inputs, the batched version is equivalent to doing attribution patching separately for each input, and then averaging - in this second version the metric per input is *not* divided by batch_size because we don't average.</details>

In [ ]:
def attr_patch_residual(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
) -> TT["component", "pos"]:
    clean_residual, residual_labels = clean_cache.accumulated_resid(
        -1, incl_mid=True, return_labels=True
    )
    corrupted_residual = corrupted_cache.accumulated_resid(
        -1, incl_mid=True, return_labels=False
    )
    corrupted_grad_residual = corrupted_grad_cache.accumulated_resid(
        -1, incl_mid=True, return_labels=False
    )
    residual_attr = einops.reduce(
        corrupted_grad_residual * (clean_residual - corrupted_residual),
        "component batch pos d_model -> component pos",
        "sum",
    )
    return residual_attr, residual_labels


residual_attr, residual_labels = attr_patch_residual(
    clean_cache, corrupted_cache, corrupted_grad_cache
)
imshow(
    residual_attr,
    y=residual_labels,
    yaxis="Component",
    xaxis="Position",
    title="Residual Attribution Patching",
)

# ### Layer Output Patching

In [ ]:
def attr_patch_layer_out(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
) -> TT["component", "pos"]:
    clean_layer_out, labels = clean_cache.decompose_resid(-1, return_labels=True)
    corrupted_layer_out = corrupted_cache.decompose_resid(-1, return_labels=False)
    corrupted_grad_layer_out = corrupted_grad_cache.decompose_resid(
        -1, return_labels=False
    )
    layer_out_attr = einops.reduce(
        corrupted_grad_layer_out * (clean_layer_out - corrupted_layer_out),
        "component batch pos d_model -> component pos",
        "sum",
    )
    return layer_out_attr, labels


layer_out_attr, layer_out_labels = attr_patch_layer_out(
    clean_cache, corrupted_cache, corrupted_grad_cache
)
imshow(
    layer_out_attr,
    y=layer_out_labels,
    yaxis="Component",
    xaxis="Position",
    title="Layer Output Attribution Patching",
)

In [ ]:
def attr_patch_head_out(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
) -> TT["component", "pos"]:
    labels = HEAD_NAMES

    clean_head_out = clean_cache.stack_head_results(-1, return_labels=False)
    corrupted_head_out = corrupted_cache.stack_head_results(-1, return_labels=False)
    corrupted_grad_head_out = corrupted_grad_cache.stack_head_results(
        -1, return_labels=False
    )
    head_out_attr = einops.reduce(
        corrupted_grad_head_out * (clean_head_out - corrupted_head_out),
        "component batch pos d_model -> component pos",
        "sum",
    )
    return head_out_attr, labels


head_out_attr, head_out_labels = attr_patch_head_out(
    clean_cache, corrupted_cache, corrupted_grad_cache
)
imshow(
    head_out_attr,
    y=head_out_labels,
    yaxis="Component",
    xaxis="Position",
    title="Head Output Attribution Patching",
)
sum_head_out_attr = einops.reduce(
    head_out_attr,
    "(layer head) pos -> layer head",
    "sum",
    layer=model.cfg.n_layers,
    head=model.cfg.n_heads,
)
imshow(
    sum_head_out_attr,
    yaxis="Layer",
    xaxis="Head Index",
    title="Head Output Attribution Patching Sum Over Pos",
)

 ### Head Activation Patching
 Intuitively, a head has three inputs, keys, queries and values. We can patch each of these individually to get a sense for where the important part of each head's input comes from!
 As a sanity check, we also do this for the mixed value. The result is a linear map of this (`z @ W_O == result`), so this is the same as patching the output of the head.
 We plot both the patch for each head over each position, and summed over position (it tends to be pretty sparse, so the latter is the same)

In [ ]:
from typing_extensions import Literal


def stack_head_vector_from_cache(
    cache, activation_name: Literal["q", "k", "v", "z"]
) -> TT["layer_and_head_index", "batch", "pos", "d_head"]:
    """Stacks the head vectors from the cache from a specific activation (key, query, value or mixed_value (z)) into a single tensor."""
    stacked_head_vectors = torch.stack(
        [cache[activation_name, l] for l in range(model.cfg.n_layers)], dim=0
    )
    stacked_head_vectors = einops.rearrange(
        stacked_head_vectors,
        "layer batch pos head_index d_head -> (layer head_index) batch pos d_head",
    )
    return stacked_head_vectors


def attr_patch_head_vector(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
    activation_name: Literal["q", "k", "v", "z"],
) -> TT["component", "pos"]:
    labels = HEAD_NAMES

    clean_head_vector = stack_head_vector_from_cache(clean_cache, activation_name)
    corrupted_head_vector = stack_head_vector_from_cache(
        corrupted_cache, activation_name
    )
    corrupted_grad_head_vector = stack_head_vector_from_cache(
        corrupted_grad_cache, activation_name
    )
    head_vector_attr = einops.reduce(
        corrupted_grad_head_vector * (clean_head_vector - corrupted_head_vector),
        "component batch pos d_head -> component pos",
        "sum",
    )
    return head_vector_attr, labels


head_vector_attr_dict = {}
for activation_name, activation_name_full in [
    ("k", "Key"),
    ("q", "Query"),
    ("v", "Value"),
    ("z", "Mixed Value"),
]:
    display(Markdown(f"#### {activation_name_full} Head Vector Attribution Patching"))
    head_vector_attr_dict[activation_name], head_vector_labels = attr_patch_head_vector(
        clean_cache, corrupted_cache, corrupted_grad_cache, activation_name
    )
    imshow(
        head_vector_attr_dict[activation_name],
        y=head_vector_labels,
        yaxis="Component",
        xaxis="Position",
        title=f"{activation_name_full} Attribution Patching",
    )
    sum_head_vector_attr = einops.reduce(
        head_vector_attr_dict[activation_name],
        "(layer head) pos -> layer head",
        "sum",
        layer=model.cfg.n_layers,
        head=model.cfg.n_heads,
    )
    imshow(
        sum_head_vector_attr,
        yaxis="Layer",
        xaxis="Head Index",
        title=f"{activation_name_full} Attribution Patching Sum Over Pos",
    )

#### Key Head Vector Attribution Patching

#### Query Head Vector Attribution Patching

#### Value Head Vector Attribution Patching

#### Mixed Value Head Vector Attribution Patching

In [ ]:
from typing_extensions import Literal


def stack_head_pattern_from_cache(
    cache,
) -> TT["layer_and_head_index", "batch", "dest_pos", "src_pos"]:
    """Stacks the head patterns from the cache into a single tensor."""
    stacked_head_pattern = torch.stack(
        [cache["pattern", l] for l in range(model.cfg.n_layers)], dim=0
    )
    stacked_head_pattern = einops.rearrange(
        stacked_head_pattern,
        "layer batch head_index dest_pos src_pos -> (layer head_index) batch dest_pos src_pos",
    )
    return stacked_head_pattern


def attr_patch_head_pattern(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
) -> TT["component", "dest_pos", "src_pos"]:
    labels = HEAD_NAMES

    clean_head_pattern = stack_head_pattern_from_cache(clean_cache)
    corrupted_head_pattern = stack_head_pattern_from_cache(corrupted_cache)
    corrupted_grad_head_pattern = stack_head_pattern_from_cache(corrupted_grad_cache)
    head_pattern_attr = einops.reduce(
        corrupted_grad_head_pattern * (clean_head_pattern - corrupted_head_pattern),
        "component batch dest_pos src_pos -> component dest_pos src_pos",
        "sum",
    )
    return head_pattern_attr, labels


head_pattern_attr, labels = attr_patch_head_pattern(
    clean_cache, corrupted_cache, corrupted_grad_cache
)

plot_attention_attr(
    einops.rearrange(
        head_pattern_attr,
        "(layer head) dest src -> layer head dest src",
        layer=model.cfg.n_layers,
        head=model.cfg.n_heads,
    ),
    clean_tokens,
    index=0,
    title="Head Pattern Attribution Patching",
)

### Head Pattern Attribution Patching

In [ ]:
def get_head_vector_grad_input_from_grad_cache(
    grad_cache: ActivationCache, activation_name: Literal["q", "k", "v"], layer: int
) -> TT["batch", "pos", "head_index", "d_model"]:
    vector_grad = grad_cache[activation_name, layer]
    ln_scales = grad_cache["scale", layer, "ln1"]
    attn_layer_object = model.blocks[layer].attn

    if activation_name == "q":
      W = attn_layer_object.W_Q
    elif activation_name == "k":
      W = attn_layer_object.W_K
    elif activation_name == "v":
      W = attn_layer_object.W_V
    else:
      raise ValueError("Invalid activation name")

    return einsum(
        "batch pos head_index d_head, batch pos, head_index d_model d_head -> batch pos head_index d_model",
        vector_grad,
        ln_scales.squeeze(-1),
        W,
    )


def get_stacked_head_vector_grad_input(
    grad_cache, activation_name: Literal["q", "k", "v"]
) -> TT["layer", "batch", "pos", "head_index", "d_model"]:
    return torch.stack(
        [
            get_head_vector_grad_input_from_grad_cache(grad_cache, activation_name, l)
            for l in range(model.cfg.n_layers)
        ],
        dim=0,
    )


def get_full_vector_grad_input(
    grad_cache,
) -> TT["qkv", "layer", "batch", "pos", "head_index", "d_model"]:
    return torch.stack(
        [
            get_stacked_head_vector_grad_input(grad_cache, activation_name)
            for activation_name in ["q", "k", "v"]
        ],
        dim=0,
    )


def attr_patch_head_path(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
) -> TT["qkv", "dest_component", "src_component", "pos"]:
    """
    Computes the attribution patch along the path between each pair of heads.

    Sets this to zero for the path from any late head to any early head

    """
    start_labels = HEAD_NAMES
    end_labels = HEAD_NAMES_QKV
    full_vector_grad_input = get_full_vector_grad_input(corrupted_grad_cache)
    clean_head_result_stack = clean_cache.stack_head_results(-1)
    corrupted_head_result_stack = corrupted_cache.stack_head_results(-1)
    diff_head_result = einops.rearrange(
        clean_head_result_stack - corrupted_head_result_stack,
        "(layer head_index) batch pos d_model -> layer batch pos head_index d_model",
        layer=model.cfg.n_layers,
        head_index=model.cfg.n_heads,
    )
    path_attr = einsum(
        "qkv layer_end batch pos head_end d_model, layer_start batch pos head_start d_model -> qkv layer_end head_end layer_start head_start pos",
        full_vector_grad_input,
        diff_head_result,
    )
    correct_layer_order_mask = (
        torch.arange(model.cfg.n_layers)[None, :, None, None, None, None]
        > torch.arange(model.cfg.n_layers)[None, None, None, :, None, None]
    ).to(path_attr.device)
    zero = torch.zeros(1, device=path_attr.device)
    path_attr = torch.where(correct_layer_order_mask, path_attr, zero)

    path_attr = einops.rearrange(
        path_attr,
        "qkv layer_end head_end layer_start head_start pos -> (layer_end head_end qkv) (layer_start head_start) pos",
    )
    return path_attr, end_labels, start_labels


head_path_attr, end_labels, start_labels = attr_patch_head_path(
    clean_cache, corrupted_cache, corrupted_grad_cache
)

imshow(
    head_path_attr.sum(-1),
    y=end_labels,
    yaxis="Path End (Head Input)",
    x=start_labels,
    xaxis="Path Start (Head Output)",
    title="Head Path Attribution Patching",
)

ValueError: Einstein sum subscript 'bp' does not contain the correct number of indices for operand 1.

 This is hard to parse. Here's an experiment with filtering for the most important heads and showing their paths.

In [ ]:
head_out_values, head_out_indices = head_out_attr.sum(-1).abs().sort(descending=True)
line(head_out_values)
top_head_indices = head_out_indices[:22].sort().values
top_end_indices = []
top_end_labels = []
top_start_indices = []
top_start_labels = []
for i in top_head_indices:
  i = i.item()
  top_start_indices.append(i)
  top_start_labels.append(start_labels[i])

  for j in range(3):
    top_end_indices.append(3 * i + j)
    top_end_labels.append(end_labels[3 * i + j])

imshow(
    head_path_attr[top_end_indices, :][:, top_start_indices].sum(-1),
    y=top_end_labels,
    yaxis="Path End (Head Input)",
    x=top_start_labels,
    xaxis="Path Start (Head Output)",
    title="Head Path Attribution Patching (Filtered for Top Heads)",
)

NameError: name 'start_labels' is not defined

In [ ]:
for j, composition_type in enumerate(["Query", "Key", "Value"]):
    imshow(
        head_path_attr[top_end_indices, :][:, top_start_indices][j::3].sum(-1),
        y=top_end_labels[j::3],
        yaxis="Path End (Head Input)",
        x=top_start_labels,
        xaxis="Path Start (Head Output)",
        title=f"Head Path to {composition_type} Attribution Patching (Filtered for Top Heads)",
    )

In [ ]:
top_head_path_attr = einops.rearrange(
    head_path_attr[top_end_indices, :][:, top_start_indices].sum(-1),
    "(head_end qkv) head_start -> qkv head_end head_start",
    qkv=3,
)
imshow(
    top_head_path_attr,
    y=[i[:-1] for i in top_end_labels[::3]],
    yaxis="Path End (Head Input)",
    x=top_start_labels,
    xaxis="Path Start (Head Output)",
    title=f"Head Path Attribution Patching (Filtered for Top Heads)",
    facet_col=0,
    facet_labels=["Query", "Key", "Value"],
)

 Let's now dive into 3 interesting heads: L5H5 (induction head), L8H6 (S-Inhibition Head), L9H9 (Name Mover) and look at their input and output paths (note - Q input means )

In [ ]:
interesting_heads = [
    5 * model.cfg.n_heads + 5,
    8 * model.cfg.n_heads + 6,
    9 * model.cfg.n_heads + 9,
]
interesting_head_labels = [HEAD_NAMES[i] for i in interesting_heads]
for head_index, label in zip(interesting_heads, interesting_head_labels):
    in_paths = head_path_attr[3 * head_index : 3 * head_index + 3].sum(-1)
    out_paths = head_path_attr[:, head_index].sum(-1)
    out_paths = einops.rearrange(out_paths, "(layer_head qkv) -> qkv layer_head", qkv=3)
    all_paths = torch.cat([in_paths, out_paths], dim=0)
    all_paths = einops.rearrange(
        all_paths,
        "path_type (layer head) -> path_type layer head",
        layer=model.cfg.n_layers,
        head=model.cfg.n_heads,
    )
    imshow(
        all_paths,
        facet_col=0,
        facet_labels=[
            "Query (In)",
            "Key (In)",
            "Value (In)",
            "Query (Out)",
            "Key (Out)",
            "Value (Out)",
        ],
        title=f"Input and Output Paths for head {label}",
        yaxis="Layer",
        xaxis="Head",
    )

 ## Validating Attribution vs Activation Patching
 Let's now compare attribution and activation patching. Generally it's a decent approximation! The main place it fails is MLP0 and the residual stream
 My fuzzy intuition is that attribution patching works badly for "big" things which are poorly modelled as linear approximations, and works well for "small" things which are more like incremental changes. Anything involving replacing the embedding is a "big" thing, which includes residual streams, and in GPT-2 small MLP0 seems to be used as an "extended embedding" (where later layers use MLP0's output instead of the token embedding), so I also count it as big.
 See more discussion in the accompanying blog post!


 First do some refactoring to make attribution patching more generic. We make an attribution cache, which is an ActivationCache where each element is (clean_act - corrupted_act) * corrupted_grad, so that it's the per-element attribution for each activation. Thanks to linearity, we just compute things by adding stuff up along the relevant dimensions!

In [ ]:
attribution_cache_dict = {}
for key in corrupted_grad_cache.cache_dict.keys():
    attribution_cache_dict[key] = corrupted_grad_cache.cache_dict[key] * (
        clean_cache.cache_dict[key] - corrupted_cache.cache_dict[key]
    )
attr_cache = ActivationCache(attribution_cache_dict, model)

 By block: For each head we patch the starting residual stream, attention output + MLP output

In [ ]:
str_tokens = model.to_str_tokens(clean_tokens[0])
context_length = len(str_tokens)

In [ ]:
every_block_act_patch_result = patching.get_act_patch_block_every(
    model, corrupted_tokens, clean_cache, ioi_metric
)
imshow(
    every_block_act_patch_result,
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    zmax=1,
    zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
)

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

In [ ]:
def get_attr_patch_block_every(attr_cache):
    resid_pre_attr = einops.reduce(
        attr_cache.stack_activation("resid_pre"),
        "layer batch pos d_model -> layer pos",
        "sum",
    )
    attn_out_attr = einops.reduce(
        attr_cache.stack_activation("attn_out"),
        "layer batch pos d_model -> layer pos",
        "sum",
    )
    mlp_out_attr = einops.reduce(
        attr_cache.stack_activation("mlp_out"),
        "layer batch pos d_model -> layer pos",
        "sum",
    )

    every_block_attr_patch_result = torch.stack(
        [resid_pre_attr, attn_out_attr, mlp_out_attr], dim=0
    )
    return every_block_attr_patch_result


every_block_attr_patch_result = get_attr_patch_block_every(attr_cache)
imshow(
    every_block_attr_patch_result,
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Attribution Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    zmax=1,
    zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
)

In [ ]:
scatter(
    y=every_block_attr_patch_result.reshape(3, -1),
    x=every_block_act_patch_result.reshape(3, -1),
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Attribution vs Activation Patching Per Block",
    xaxis="Activation Patch",
    yaxis="Attribution Patch",
    hover=[
        f"Layer {l}, Position {p}, |{str_tokens[p]}|"
        for l in range(model.cfg.n_layers)
        for p in range(context_length)
    ],
    color=einops.repeat(
        torch.arange(model.cfg.n_layers), "layer -> (layer pos)", pos=context_length
    ),
    color_continuous_scale="Portland",
)

 By head: For each head we patch the output, query, key, value or pattern. We do all positions at once so it's not super slow.

In [ ]:
every_head_all_pos_act_patch_result = patching.get_act_patch_attn_head_all_pos_every(
    model, corrupted_tokens, clean_cache, ioi_metric
)
imshow(
    every_head_all_pos_act_patch_result,
    facet_col=0,
    facet_labels=["Output", "Query", "Key", "Value", "Pattern"],
    title="Activation Patching Per Head (All Pos)",
    xaxis="Head",
    yaxis="Layer",
    zmax=1,
    zmin=-1,
)

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

In [ ]:
def get_attr_patch_attn_head_all_pos_every(attr_cache):
    head_out_all_pos_attr = einops.reduce(
        attr_cache.stack_activation("z"),
        "layer batch pos head_index d_head -> layer head_index",
        "sum",
    )
    head_q_all_pos_attr = einops.reduce(
        attr_cache.stack_activation("q"),
        "layer batch pos head_index d_head -> layer head_index",
        "sum",
    )
    head_k_all_pos_attr = einops.reduce(
        attr_cache.stack_activation("k"),
        "layer batch pos head_index d_head -> layer head_index",
        "sum",
    )
    head_v_all_pos_attr = einops.reduce(
        attr_cache.stack_activation("v"),
        "layer batch pos head_index d_head -> layer head_index",
        "sum",
    )
    head_pattern_all_pos_attr = einops.reduce(
        attr_cache.stack_activation("pattern"),
        "layer batch head_index dest_pos src_pos -> layer head_index",
        "sum",
    )

    return torch.stack(
        [
            head_out_all_pos_attr,
            head_q_all_pos_attr,
            head_k_all_pos_attr,
            head_v_all_pos_attr,
            head_pattern_all_pos_attr,
        ]
    )


every_head_all_pos_attr_patch_result = get_attr_patch_attn_head_all_pos_every(
    attr_cache
)
imshow(
    every_head_all_pos_attr_patch_result,
    facet_col=0,
    facet_labels=["Output", "Query", "Key", "Value", "Pattern"],
    title="Attribution Patching Per Head (All Pos)",
    xaxis="Head",
    yaxis="Layer",
    zmax=1,
    zmin=-1,
)

In [ ]:
scatter(
    y=every_head_all_pos_attr_patch_result.reshape(5, -1),
    x=every_head_all_pos_act_patch_result.reshape(5, -1),
    facet_col=0,
    facet_labels=["Output", "Query", "Key", "Value", "Pattern"],
    title="Attribution vs Activation Patching Per Head (All Pos)",
    xaxis="Activation Patch",
    yaxis="Attribution Patch",
    include_diag=True,
    hover=head_out_labels,
    color=einops.repeat(
        torch.arange(model.cfg.n_layers),
        "layer -> (layer head)",
        head=model.cfg.n_heads,
    ),
    color_continuous_scale="Portland",
)

 We see pretty good results in general, but significant errors for heads L5H5 on query and moderate errors for head L10H7 on query and key, and moderate errors for head L11H10 on key. But each of these is fine for pattern and output. My guess is that the problem is that these have pretty saturated attention on a single token, and the linear approximation is thus not great on the attention calculation here, but I'm not sure. When we plot the attention patterns, we do see this!
 Note that the axis labels are for the *first* prompt's tokens, but each facet is a different prompt, so this is somewhat inaccurate. In particular, every odd facet has indirect object and subject in the opposite order (IO first). But otherwise everything lines up between the prompts

In [ ]:
graph_tok_labels = [
    f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))
]
imshow(
    clean_cache["pattern", 5][:, 5],
    x=graph_tok_labels,
    y=graph_tok_labels,
    facet_col=0,
    title="Attention for Head L5H5",
    facet_name="Prompt",
)
imshow(
    clean_cache["pattern", 10][:, 7],
    x=graph_tok_labels,
    y=graph_tok_labels,
    facet_col=0,
    title="Attention for Head L10H7",
    facet_name="Prompt",
)
imshow(
    clean_cache["pattern", 11][:, 10],
    x=graph_tok_labels,
    y=graph_tok_labels,
    facet_col=0,
    title="Attention for Head L11H10",
    facet_name="Prompt",
)


# [markdown]

In [ ]:
every_head_by_pos_act_patch_result = patching.get_act_patch_attn_head_by_pos_every(
    model, corrupted_tokens, clean_cache, ioi_metric
)
every_head_by_pos_act_patch_result = einops.rearrange(
    every_head_by_pos_act_patch_result,
    "act_type layer pos head -> act_type (layer head) pos",
)
imshow(
    every_head_by_pos_act_patch_result,
    facet_col=0,
    facet_labels=["Output", "Query", "Key", "Value", "Pattern"],
    title="Activation Patching Per Head (By Pos)",
    xaxis="Position",
    yaxis="Layer & Head",
    zmax=1,
    zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
    y=head_out_labels,
)

  0%|          | 0/2160 [00:00<?, ?it/s]

  0%|          | 0/2160 [00:00<?, ?it/s]

  0%|          | 0/2160 [00:00<?, ?it/s]

  0%|          | 0/2160 [00:00<?, ?it/s]

  0%|          | 0/2160 [00:00<?, ?it/s]

In [ ]:
def get_attr_patch_attn_head_by_pos_every(attr_cache):
    head_out_by_pos_attr = einops.reduce(
        attr_cache.stack_activation("z"),
        "layer batch pos head_index d_head -> layer pos head_index",
        "sum",
    )
    head_q_by_pos_attr = einops.reduce(
        attr_cache.stack_activation("q"),
        "layer batch pos head_index d_head -> layer pos head_index",
        "sum",
    )
    head_k_by_pos_attr = einops.reduce(
        attr_cache.stack_activation("k"),
        "layer batch pos head_index d_head -> layer pos head_index",
        "sum",
    )
    head_v_by_pos_attr = einops.reduce(
        attr_cache.stack_activation("v"),
        "layer batch pos head_index d_head -> layer pos head_index",
        "sum",
    )
    head_pattern_by_pos_attr = einops.reduce(
        attr_cache.stack_activation("pattern"),
        "layer batch head_index dest_pos src_pos -> layer dest_pos head_index",
        "sum",
    )

    return torch.stack(
        [
            head_out_by_pos_attr,
            head_q_by_pos_attr,
            head_k_by_pos_attr,
            head_v_by_pos_attr,
            head_pattern_by_pos_attr,
        ]
    )


every_head_by_pos_attr_patch_result = get_attr_patch_attn_head_by_pos_every(attr_cache)
every_head_by_pos_attr_patch_result = einops.rearrange(
    every_head_by_pos_attr_patch_result,
    "act_type layer pos head -> act_type (layer head) pos",
)
imshow(
    every_head_by_pos_attr_patch_result,
    facet_col=0,
    facet_labels=["Output", "Query", "Key", "Value", "Pattern"],
    title="Attribution Patching Per Head (By Pos)",
    xaxis="Position",
    yaxis="Layer & Head",
    zmax=1,
    zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
    y=head_out_labels,
)

In [ ]:
scatter(
    y=every_head_by_pos_attr_patch_result.reshape(5, -1),
    x=every_head_by_pos_act_patch_result.reshape(5, -1),
    facet_col=0,
    facet_labels=["Output", "Query", "Key", "Value", "Pattern"],
    title="Attribution vs Activation Patching Per Head (by Pos)",
    xaxis="Activation Patch",
    yaxis="Attribution Patch",
    include_diag=True,
    hover=[f"{label} {tok}" for label in head_out_labels for tok in graph_tok_labels],
    color=einops.repeat(
        torch.arange(model.cfg.n_layers),
        "layer -> (layer head pos)",
        head=model.cfg.n_heads,
        pos=15,
    ),
    color_continuous_scale="Portland",
)

 ## Factual Knowledge Patching Example
 Incomplete, but maybe of interest!
 Note that I have better results with the corrupted prompt as having random words rather than Colosseum.

In [ ]:
gpt2_xl = HookedTransformer.from_pretrained("gpt2-xl")
clean_prompt = "The Eiffel Tower is located in the city of"
clean_answer = " Paris"
# corrupted_prompt = "The red brown fox jumps is located in the city of"
corrupted_prompt = "The Colosseum is located in the city of"
corrupted_answer = " Rome"
utils.test_prompt(clean_prompt, clean_answer, gpt2_xl)
utils.test_prompt(corrupted_prompt, corrupted_answer, gpt2_xl)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-xl into HookedTransformer
Tokenized prompt: ['<|endoftext|>', 'The', ' E', 'iff', 'el', ' Tower', ' is', ' located', ' in', ' the', ' city', ' of']
Tokenized answer: [' Paris']


Performance on answer token:
Rank: 0        Logit: 20.73 Prob: 95.80% Token: | Paris|

Top 0th token. Logit: 20.73 Prob: 95.80% Token: | Paris|
Top 1th token. Logit: 16.49 Prob:  1.39% Token: | E|
Top 2th token. Logit: 14.69 Prob:  0.23% Token: | the|
Top 3th token. Logit: 14.58 Prob:  0.21% Token: | É|
Top 4th token. Logit: 14.44 Prob:  0.18% Token: | France|
Top 5th token. Logit: 14.36 Prob:  0.16% Token: | Mont|
Top 6th token. Logit: 13.77 Prob:  0.09% Token: | Le|
Top 7th token. Logit: 13.66 Prob:  0.08% Token: | Ang|
Top 8th token. Logit: 13.43 Prob:  0.06% Token: | V|
Top 9th token. Logit: 13.42 Prob:  0.06% Token: | Stras|


Ranks of the answer tokens: [(' Paris', 0)]

Tokenized prompt: ['<|endoftext|>', 'The', ' Col', 'os', 'se', 'um', ' is', ' located', ' in', ' the', ' city', ' of']
Tokenized answer: [' Rome']


Performance on answer token:
Rank: 0        Logit: 20.02 Prob: 83.70% Token: | Rome|

Top 0th token. Logit: 20.02 Prob: 83.70% Token: | Rome|
Top 1th token. Logit: 17.03 Prob:  4.23% Token: | Naples|
Top 2th token. Logit: 16.85 Prob:  3.51% Token: | Pompe|
Top 3th token. Logit: 16.14 Prob:  1.73% Token: | Ver|
Top 4th token. Logit: 15.87 Prob:  1.32% Token: | Florence|
Top 5th token. Logit: 14.77 Prob:  0.44% Token: | Roma|
Top 6th token. Logit: 14.68 Prob:  0.40% Token: | Milan|
Top 7th token. Logit: 14.66 Prob:  0.39% Token: | ancient|
Top 8th token. Logit: 14.37 Prob:  0.29% Token: | Pal|
Top 9th token. Logit: 14.30 Prob:  0.27% Token: | Constantinople|


Ranks of the answer tokens: [(' Rome', 0)]

In [ ]:
clean_answer_index = gpt2_xl.to_single_token(clean_answer)
corrupted_answer_index = gpt2_xl.to_single_token(corrupted_answer)


def factual_logit_diff(logits: TT["batch", "position", "d_vocab"]):
    return logits[0, -1, clean_answer_index] - logits[0, -1, corrupted_answer_index]

In [ ]:
clean_logits, clean_cache = gpt2_xl.run_with_cache(clean_prompt)
CLEAN_LOGIT_DIFF_FACTUAL = factual_logit_diff(clean_logits).item()
corrupted_logits, _ = gpt2_xl.run_with_cache(corrupted_prompt)
CORRUPTED_LOGIT_DIFF_FACTUAL = factual_logit_diff(corrupted_logits).item()


def factual_metric(logits: TT["batch", "position", "d_vocab"]):
    return (factual_logit_diff(logits) - CORRUPTED_LOGIT_DIFF_FACTUAL) / (
        CLEAN_LOGIT_DIFF_FACTUAL - CORRUPTED_LOGIT_DIFF_FACTUAL
    )


print("Clean logit diff:", CLEAN_LOGIT_DIFF_FACTUAL)
print("Corrupted logit diff:", CORRUPTED_LOGIT_DIFF_FACTUAL)
print("Clean Metric:", factual_metric(clean_logits))
print("Corrupted Metric:", factual_metric(corrupted_logits))

Clean logit diff: 10.634519577026367
Corrupted logit diff: -8.988396644592285
Clean Metric: tensor(1., device='cuda:0', grad_fn=<DivBackward0>)
Corrupted Metric: tensor(0., device='cuda:0', grad_fn=<DivBackward0>)


In [ ]:
# corrupted_value, corrupted_cache, corrupted_grad_cache = get_cache_fwd_and_bwd(gpt2_xl, corrupted_prompt, factual_metric)

In [ ]:
clean_tokens = gpt2_xl.to_tokens(clean_prompt)
clean_str_tokens = gpt2_xl.to_str_tokens(clean_prompt)
corrupted_tokens = gpt2_xl.to_tokens(corrupted_prompt)
corrupted_str_tokens = gpt2_xl.to_str_tokens(corrupted_prompt)
print("Clean:", clean_str_tokens)
print("Corrupted:", corrupted_str_tokens)

Clean: ['<|endoftext|>', 'The', ' E', 'iff', 'el', ' Tower', ' is', ' located', ' in', ' the', ' city', ' of']
Corrupted: ['<|endoftext|>', 'The', ' Col', 'os', 'se', 'um', ' is', ' located', ' in', ' the', ' city', ' of']


In [ ]:
def act_patch_residual(clean_cache, corrupted_tokens, model: HookedTransformer, metric):
    if len(corrupted_tokens.shape) == 2:
        corrupted_tokens = corrupted_tokens[0]
    residual_patches = torch.zeros(
        (model.cfg.n_layers, len(corrupted_tokens)), device=model.cfg.device
    )

    def residual_hook(resid_pre, hook, layer, pos):
        resid_pre[:, pos, :] = clean_cache["resid_pre", layer][:, pos, :]
        return resid_pre

    for layer in tqdm.tqdm(range(model.cfg.n_layers)):
        for pos in range(len(corrupted_tokens)):
            patched_logits = model.run_with_hooks(
                corrupted_tokens,
                fwd_hooks=[
                    (
                        f"blocks.{layer}.hook_resid_pre",
                        partial(residual_hook, layer=layer, pos=pos),
                    )
                ],
            )
            residual_patches[layer, pos] = metric(patched_logits).item()
    return residual_patches


residual_act_patch = act_patch_residual(
    clean_cache, corrupted_tokens, gpt2_xl, factual_metric
)

imshow(
    residual_act_patch,
    title="Factual Recall Patching (Residual)",
    xaxis="Position",
    yaxis="Layer",
    x=clean_str_tokens,
)

  0%|          | 0/48 [00:00<?, ?it/s]